In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when

In [0]:
# Inicia o Spark
spark = SparkSession.builder.appName("BikeClima-Ouro").getOrCreate()

# Caminho base da camada bronze
caminho_bike_bronze_dir = "/Volumes/projeto/default/bronze/"

# Lista os arquivos no diretório bronze
arquivos_bronze = dbutils.fs.ls(caminho_bike_bronze_dir)

# Filtra arquivos de bike que começam com "bike_" e contêm ".parquet"
bike_arquivos = [f for f in arquivos_bronze if f.name.startswith("bike_") and "parquet" in f.name]

# Ordena do mais recente para o mais antigo
bike_arquivos_ordenados = sorted(bike_arquivos, key=lambda f: f.modificationTime, reverse=True)

if bike_arquivos_ordenados:
    caminho_bike_bronze = bike_arquivos_ordenados[0].path
    print(f"Arquivo de bike mais recente: {caminho_bike_bronze}")

    # Caminhos fixos
    caminho_clima_prata = "/Volumes/projeto/default/prata/climasp.parquet"
    caminho_ouro = "/Volumes/projeto/default/ouro/bike_clima.parquet"

    # Lendo os dados
    df_bikes = spark.read.parquet(caminho_bike_bronze)
    df_clima = spark.read.parquet(caminho_clima_prata)

    # Pegando a linha única do clima
    clima = df_clima.collect()[0]

    # Juntando com os dados de clima
    df_bike_clima = df_bikes \
        .withColumn("temperature", lit(clima["temperature"])) \
        .withColumn("real_temperature", lit(clima["real_temperature"])) \
        .withColumn("temp_min", lit(clima["temp_min"])) \
        .withColumn("temp_max", lit(clima["temp_max"])) \
        .withColumn("humidity", lit(clima["humidity"])) \
        .withColumn("description", lit(clima["weather_description"])) \
        .withColumn("warning",
            when((lit(clima["temperature"]) > 35) & (lit(clima["humidity"]) < 30), "Não seguro")
            .when((lit(clima["temperature"]) > 30) & (lit(clima["humidity"]) < 40), "Segurança baixa")
            .when((lit(clima["temperature"]) > 25) & (lit(clima["humidity"]) < 60), "Segurança média")
            .otherwise("Segurança alta")
        )

    # Salvando na camada ouro
    df_bike_clima.write.mode("overwrite").parquet(caminho_ouro)
    df_bike_clima.write.mode("overwrite").saveAsTable("bike_clima_tabela")
else:
    print("Nenhum arquivo 'bike_*.parquet' encontrado na camada bronze.")
